# LLaMA_Factory (HPC)

## 初始環境設定, 切換到 kernel pytorch_2.1.0-cuda11.8-cudnn8-devel.sif

In [ ]:
# 初始環境設定
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.local/bin:/usr/ubuntu_bin'
os.environ['PATH']=os.environ['PATH']+':'+Add_Binarry_Path

## 確認CUDA版本, 以及否能使用GPU

In [ ]:
!nvidia-smi
import torch
torch.cuda.is_available()

## 可能需要切換到原生 python3 kernel

In [ ]:
%%bash
rm -rf LLaMA-Factory
git clone https://github.com/hiyouga/LLaMA-Factory.git

## 安裝套件, 切換到 kernel pytorch_2.1.0-cuda11.8-cudnn8-devel.sif

In [ ]:
%%bash
#rm -rf LLaMA-Factory
#git clone https://github.com/hiyouga/LLaMA-Factory.git
cd ./LLaMA-Factory
pip install -r requirements.txt -q
pip install bitsandbytes==0.42.0 -q

## 撰寫程式碼

In [ ]:
%%bash
# IP
node_ip=$(cat /etc/hosts |grep "$(hostname -a)" | awk '{print $1}')
# PORT
noed_port_genai=$(python -c "import socket; s = socket.socket(socket.AF_INET, socket.SOCK_STREAM); s.bind(('', 0)); addr = s.getsockname(); s.close(); print(addr[1])")
# PROXY
proxy_url=/rstudio/${node_ip}/${noed_port_genai}
# URL
https_url=https://node01.biobank.org.tw${proxy_url}/

# SCRIPT FILE
cat << EOF >  LLaMA-Factory/src/train_web_demo.py
from llmtuner import create_ui
import os
os.environ["WANDB_DISABLED"] = "true"

print("${https_url}")

def main():
    demo = create_ui()
    demo.queue()
    demo.launch(server_port=${noed_port_genai}, server_name="$(hostname -s)", share=False, inbrowser=True, root_path="${proxy_url}", auth=("nchc", "nchcorgtw"))

if __name__ == "__main__":
    main()
EOF

## 強制刪除運行中服務

In [ ]:
# 強制刪除運行中服務
!ps -ef |grep train_web_demo.py | awk '{print $2}' | xargs kill -9

## 開啟WebUI
- model: /work/u00cjz00/slurm_jobs/github/models/Llama-2-7b-chat-hf

In [ ]:
%%bash
#請更新以下 HF_TOKEN
cd LLaMA-Factory
CUDA_VISIBLE_DEVICES=0 HF_TOKEN='hf_xxxxxxxxxxxxxxxxxx' python src/train_web_demo.py

# 上傳輸出的檔案到 HuggingFace

In [ ]:
!pip install huggingface_hub

## 使用API Token登入

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## 上傳模型
執行本儲存格後填入 API Token(需要有 Write 權限)，然後按下登入

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

model_id = "username/fine-tune-model" #@param {type:"string"}
export_folder_path = "/content/LLaMA-Factory/export" #@param {type:"string"}
api.create_repo(model_id, private=True, exist_ok=True, repo_type="model")
api.upload_folder(
    folder_path=export_folder_path,
    repo_id=model_id,
)